# LAQN Dataset Assesment


1) I'll be start adding my main paths and modules I will be using in this notebook below.

In [72]:
# possible python modules i will be using below
from curses import meta
import os
import pandas as pd
from pathlib import Path
import csv
from collections import defaultdict
#function 7 importing the full analysis function from pollution_analysis
import sys
sys.path.append('/mnt/user-data/outputs')

#last detailed anlasye and visualization imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualisation style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

#findings 7 .func
# for parse pdf uk pollutant limitations to csv
import re
# pdfplumber for pdf parsing

# function 5. chi-square test
from scipy import stats

#define base path  without hardcoding
base_dir = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "optimased"
#metadata file for pollutant name, location and site names
metadata_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" /"laqn"/"optimased_siteSpecies.csv"

# output path for saving statistics 1. function
#the first analyse dataset created without inclitiong nan optimased files, and cross referencing that's why changed the name to dataset_statistics-noNAN-incl.csv
os.makedirs(base_dir / "report", exist_ok=True)
stats_output_path =  Path.home()/"Desktop" / "data science projects" / "air-pollution-levels" / "data" /"laqn"/"report"/ "laqn_stats.csv"

# output paths for saving all the pollutant distribution and nan value analysis.
pollutant_distrubution_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" /"laqn" / "report"/"pollutant_distribution.csv"
nan_val_pollutant_split_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" /"laqn" / "report" / "nan_values_by_pollutant.csv"
nan_val_stationPollutant_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" /"laqn" / "report" / "nan_values_by_station_pollutant.csv"


# log file from nan replacement process
nan_log_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "logs" / "NaN_values_record.csv"

# function for uk pollutant regulations pdf to parse csv file path
csv_output_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "defra" / "capabilities" / "uk_pollutant_limits.csv"


# data quality metrics report output path
quality_output = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn"/ "report" / "quality_metrics_validation.csv"
quality_output.parent.mkdir(parents=True, exist_ok=True)

#chi-square test output path func 5
chi_square_output1 = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "report" / "chi_square_tests1.csv"
# save results to csv
chi_square_laqn_output = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "report" / "chi_square_tests.csv"


 # chi-square yearly comperison analyse path
year_diff_output = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "report" / "year_coverage_analysis.csv"



# detailed last analysis and visualization output directory
report_dir = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "report" / "detailed_analysis"

report_dir.mkdir(parents=True, exist_ok=True)

## 1) Initial Dataset Assessment: Raw Numbers

Before conducting quality checks, I need to establish the baseline characteristics of the LAQN dataset. This section calculates comprehensive statistics about the data collection effort, including file counts, measurement records, station coverage, and pollutant distribution.

### Purpose
- Document the scale and scope of data collection.
- Establish baseline metrics for comparison with LAQN.
- Provide context for subsequent quality analysis.

### Methodology
The function `get_laqn_dataset_statistics()` performs the following:
1. Loads standardised metadata to identify unique stations and pollutants.
2. Counts files across all three yearly directories (2023, 2024, 2025).
3. Calculates total measurement records by reading all CSV files.
4. Determines spatial coverage from unique coordinate pairs.
5. Documents temporal coverage (35 months: January 2023 to November 2025).

### Notes
- File counting is fast (scans directory structure only).
- Record counting can be slow (reads every CSV file).
- Results are saved to csv.

In [26]:
def get_laqn_dataset_statistics(base_dir, metadata_path, nan_log_path):
    """
    Calculate statistics for the LAQN dataset using the new column structure.
    This function scans all CSV files recursively under base_dir and calculates key metrics needed for reporting.

    Parameters:
        base_dir : Path
            Base directory containing LAQN data folders.
        metadata_path : Path
            Path to the standardised metadata csv file.
        nan_log_path : Path
            Path to the NaN values log file after notice data flags, changed them to NaN.

    Returns:
        dict : Dictionary containing all calculated statistics.
    """
    stats = {}

    # read metadata to get station and pollutant info
    print("\nloading metadata...")
    metadata = pd.read_csv(metadata_path, encoding="utf-8")

    # calculate metadata statistics
    stats['unique_stations'] = metadata['SiteCode'].nunique()
    stats['total_combinations'] = len(metadata)
    stats['unique_pollutants'] = metadata['SpeciesCode'].nunique()

    # get pollutant breakdown
    pollutant_counts = metadata['SpeciesCode'].value_counts()
    stats['pollutant_distribution'] = pollutant_counts.to_dict()

    # create set of expected (SiteCode, SpeciesCode) pairs from metadata
    expected_pairs = set(
        zip(metadata['SiteCode'], metadata['SpeciesCode'])
    )
    stats['expected_pairs'] = len(expected_pairs)
    print(f"  expected SiteCode/SpeciesCode pairs from metadata: {len(expected_pairs)}")

    # count unique coordinates for spatial coverage
    unique_coords = metadata[['Latitude', 'Longitude']].drop_duplicates()
    stats['unique_locations'] = len(unique_coords)

    # Scan all CSVs in all subfolders under base_dir
    print("\nscanning optimased directory for collected data...")
    all_csv_files = list(Path(base_dir).rglob('*.csv'))
    total_files = len(all_csv_files)
    print(f"\nTotal CSV files found: {total_files}")
    stats['total_files'] = total_files

    # Count files, records, and missing values by period (e.g., "2023_apr")
    files_by_period = defaultdict(int)
    records_by_period = defaultdict(int)
    missing_by_period = defaultdict(int)

    all_csvs = []
    total_records = 0
    total_missing = 0

    print("\nReading all CSV files to calculate statistics...")
    for csv_file in all_csv_files:
        period = csv_file.parent.name
        try:
            df = pd.read_csv(csv_file)
            n_records = len(df)
            n_missing = df['@Value'].isna().sum() + (df['@Value'] == "").sum() if '@Value' in df.columns else 0
            all_csvs.append(df)
            files_by_period[period] += 1
            records_by_period[period] += n_records
            missing_by_period[period] += n_missing
            total_records += n_records
            total_missing += n_missing
        except Exception as e:
            print(f"  warning: could not read {csv_file.name}: {e}")

    stats['files_by_period'] = dict(files_by_period)
    stats['records_by_period'] = dict(records_by_period)
    stats['missing_by_period'] = dict(missing_by_period)
    stats['total_records'] = total_records
    stats['total_missing'] = total_missing
    stats['overall_completeness'] = ((total_records - total_missing) / total_records * 100) if total_records > 0 else 0

    for period in files_by_period:
        rec = records_by_period[period]
        miss = missing_by_period[period]
        miss_pct = (miss / rec * 100) if rec > 0 else 0
        print(f"  {period}: {files_by_period[period]} files, {rec:,} records, {miss:,} missing ({miss_pct:.2f}%)")

    # cross-reference metadata with collected data
    print("\ncross-referencing collected data with metadata...")

    if all_csvs:
        all_data = pd.concat(all_csvs, ignore_index=True)

        # check if required columns exist in csv files
        if 'SiteCode' in all_data.columns and 'SpeciesCode' in all_data.columns:
            # identify actual (SiteCode, SpeciesCode) pairs in collected data
            collected_pairs = set(
                zip(all_data['SiteCode'], all_data['SpeciesCode'])
            )
            stats['collected_pairs'] = len(collected_pairs)

            # find missing pairs (in metadata but not in collected data)
            missing_pairs = expected_pairs - collected_pairs
            stats['missing_pairs'] = list(missing_pairs)
            stats['missing_pairs_count'] = len(missing_pairs)

            # find extra pairs (in collected data but not in metadata)
            extra_pairs = collected_pairs - expected_pairs
            stats['extra_pairs'] = list(extra_pairs)
            stats['extra_pairs_count'] = len(extra_pairs)

            print(f"  expected pairs from metadata: {len(expected_pairs)}")
            print(f"  actually collected pairs: {len(collected_pairs)}")
            print(f"  missing pairs (in metadata but not collected): {len(missing_pairs)}")
            print(f"  extra pairs (collected but not in metadata): {len(extra_pairs)}")

            # group by SiteCode and SpeciesCode, count missing values
            missing_breakdown = {}
            for (site, species), group in all_data.groupby(['SiteCode', 'SpeciesCode']):
                total_rows = len(group)
                if '@Value' in group.columns:
                    missing_rows = group['@Value'].isna().sum() + (group['@Value'] == "").sum()
                else:
                    missing_rows = 0
                missing_breakdown[(site, species)] = (int(missing_rows), int(total_rows))
            stats['missing_by_station_pollutant'] = missing_breakdown
        else:
            print("  warning: SiteCode or SpeciesCode columns not found")
            stats['missing_by_station_pollutant'] = {}
            stats['collected_pairs'] = 0
            stats['missing_pairs'] = []
            stats['missing_pairs_count'] = 0
            stats['extra_pairs'] = []
            stats['extra_pairs_count'] = 0
    else:
        stats['missing_by_station_pollutant'] = {}
        stats['collected_pairs'] = 0
        stats['missing_pairs'] = list(expected_pairs)
        stats['missing_pairs_count'] = len(expected_pairs)
        stats['extra_pairs'] = []
        stats['extra_pairs_count'] = 0

    # distribution of nan by pollutant over time
    if stats['missing_by_station_pollutant']:
        pollutant_missing_summary = {}
        for (site, species), (missing, total) in stats['missing_by_station_pollutant'].items():
            if species not in pollutant_missing_summary:
                pollutant_missing_summary[species] = {'total_missing': 0, 'total_records': 0}
            pollutant_missing_summary[species]['total_missing'] += missing
            pollutant_missing_summary[species]['total_records'] += total
        for species in pollutant_missing_summary:
            total_missing = pollutant_missing_summary[species]['total_missing']
            total_records = pollutant_missing_summary[species]['total_records']
            percentage = (total_missing / total_records * 100) if total_records > 0 else 0
            pollutant_missing_summary[species]['percentage_missing'] = percentage
        stats['missing_by_pollutant_type'] = pollutant_missing_summary
    else:
        stats['missing_by_pollutant_type'] = {}

    # log file created during data cleaning process
    if Path(nan_log_path).exists():
        nan_log = pd.read_csv(nan_log_path)
        replacements_by_year = nan_log.groupby('year_folder')['invalid_flags_replaced'].sum().to_dict()
        stats['nan_replacements_by_year'] = replacements_by_year
        stats['total_nan_replacements'] = nan_log['invalid_flags_replaced'].sum()
        stats['mean_invalid_percentage'] = nan_log['percentage_invalid'].mean()
        stats['max_invalid_percentage'] = nan_log['percentage_invalid'].max()
    else:
        stats['nan_replacements_by_year'] = {}
        stats['total_nan_replacements'] = 0
        stats['mean_invalid_percentage'] = 0
        stats['max_invalid_percentage'] = 0

    # calculate temporal coverage based on the files collected
    stats['temporal_coverage'] = {
        'start_date': '2023-01-01',
        'end_date': '2025-11-19',
        'total_months': 35
    }


    def extract_year(period):
        # period is usually like '2023_apr' or '2024_jan'
        return str(period)[:4] if len(str(period)) >= 4 and str(period)[:4].isdigit() else 'unknown'

    files_by_year = defaultdict(int)
    records_by_year = defaultdict(int)
    missing_by_year = defaultdict(int)
    for period in files_by_period:
        year = extract_year(period)
        files_by_year[year] += files_by_period[period]
        records_by_year[year] += records_by_period[period]
        missing_by_year[year] += missing_by_period[period]
    stats['files_by_year'] = dict(files_by_year)
    stats['records_by_year'] = dict(records_by_year)
    stats['missing_by_year'] = dict(missing_by_year)

    return stats

In [27]:
def print_dataset_statistics(stats):
    """
    Print dataset statistics for LAQN using new column structure.

    Parameters:
        stats : dict
            returned by get_laqn_dataset_statistics().
    """

    print("\n" + "="*40)
    print("LAQN dataset statistics: initial assessment")
    print("="*40)

    print("\nScale and scope:")
    print(f"Total files collected: {stats['total_files']:,}")
    print(f"Total measurement records: {stats['total_records']:,}")
    print(f"Total missing values (@Value): {stats['total_missing']:,}")
    print(f"Overall completeness: {stats['overall_completeness']:.2f}%")
    print(f"Unique monitoring sites (SiteCode): {stats['unique_stations']}")
    print(f"Total site-species combinations: {stats['total_combinations']}")
    print(f"Unique pollutant types (SpeciesCode): {stats['unique_pollutants']}")
    print(f"Unique geographic locations: {stats['unique_locations']}")

    # data collection coverage
    print("\nData collection coverage:")
    print(f"Expected SiteCode/SpeciesCode pairs (from metadata): {stats.get('expected_pairs', 0)}")
    print(f"Actually collected pairs: {stats.get('collected_pairs', 0)}")
    print(f"Missing pairs (not collected): {stats.get('missing_pairs_count', 0)}")
    print(f"Extra pairs (not in metadata): {stats.get('extra_pairs_count', 0)}")

    if stats.get('missing_pairs_count', 0) > 0:
        print(f"\nWarning: {stats['missing_pairs_count']} SiteCode/SpeciesCode pairs from metadata were not found in collected data.")
        print("First 10 missing pairs:")
        for i, (site, species) in enumerate(stats['missing_pairs'][:10], 1):
            print(f"  {i}. {site} - {species}")

    if stats.get('extra_pairs_count', 0) > 0:
        print(f"\nNote: {stats['extra_pairs_count']} SiteCode/SpeciesCode pairs in collected data are not in metadata.")

    print("\nFiles by year:")
    for year, count in stats['files_by_year'].items():
        print(f"  {year}: {count:,} files")

    print("\nRecords by year:")
    for year, count in stats['records_by_year'].items():
        missing = stats['missing_by_year'].get(year, 0)
        missing_pct = (missing / count * 100) if count > 0 else 0
        print(f"  {year}: {count:,} records, {missing:,} missing ({missing_pct:.2f}%)")

    # adding nan value summary below
    print("\nNaN replacement summary:")
    print(f"Total invalid flags replaced: {stats['total_nan_replacements']:,}")
    print(f"Mean invalid percentage per file: {stats['mean_invalid_percentage']:.2f}%")
    print(f"Max invalid percentage: {stats['max_invalid_percentage']:.2f}%")

    # count of replacements by year
    if stats['nan_replacements_by_year']:
        print("\nReplacements by year:")
        for year_folder, count in stats['nan_replacements_by_year'].items():
            print(f"  {year_folder}: {count:,} flags replaced")

    print("\nTemporal coverage:")
    print(f"Start date: {stats['temporal_coverage']['start_date']}")
    print(f"End date: {stats['temporal_coverage']['end_date']}")
    print(f"Total months: {stats['temporal_coverage']['total_months']}")

    print("\nPollutant (SpeciesCode) distribution:")
    print("Site/species combinations by type:")
    for species, count in sorted(stats['pollutant_distribution'].items(),
                                 key=lambda x: x[1], reverse=True):
        percentage = (count / stats['total_combinations']) * 100
        print(f"  {species}: {count} ({percentage:.1f}%)")

    # missing value distribution by pollutant type
    print("\nMissing value distribution by pollutant type (SpeciesCode):")
    if stats.get('missing_by_pollutant_type'):
        # sort by percentage missing (highest first)
        sorted_species = sorted(
            stats['missing_by_pollutant_type'].items(),
            key=lambda x: x[1]['percentage_missing'],
            reverse=True
        )
        print(f"{'SpeciesCode':<20} {'total records':>15} {'missing':>12} {'% missing':>12}")
        print("-" * 60)
        for species, data in sorted_species:
            print(f"{species:<20} {data['total_records']:>15,} {data['total_missing']:>12,} {data['percentage_missing']:>11.2f}%")
    else:
        print("  No missing value distribution available.")

    # print missing values by site/species breakdown with row_number column
    print("\nMissing values by site/species (SiteCode/SpeciesCode):")
    if stats.get('missing_by_station_pollutant'):
        # prepare a sorted list by missing percentage descending
        breakdown = []
        for (site, species), (missing, total) in stats['missing_by_station_pollutant'].items():
            percent = (missing / total * 100) if total > 0 else 0
            breakdown.append((site, species, missing, total, percent))
        # sort by percentage descending and take top 20
        breakdown.sort(key=lambda x: x[4], reverse=True)
        breakdown = breakdown[:20]
        print(f"{'SiteCode':<20} {'SpeciesCode':<20} {'missing':>10} {'total_row':>12} {'% missing':>12}")
        print("-" * 60)
        for site, species, missing, total, percent in breakdown:
            print(f"{site:<20} {species:<20} {missing:>10,} {total:>12,} {percent:>11.2f}%")
    else:
        print("  No missing value breakdown available.")

In [28]:
# Run the analysis
stats = get_laqn_dataset_statistics(base_dir, metadata_path, nan_log_path)
print_dataset_statistics(stats)

# Save statistics for later use as csv
# Prepare flat data structure for csv
stats_rows = []
stats_rows.append(["metric", "value"])
stats_rows.append(["total_files", stats['total_files']])
stats_rows.append(["total_records", stats['total_records']])
stats_rows.append(["total_missing", stats['total_missing']])
stats_rows.append(["overall_completeness_pct", f"{stats['overall_completeness']:.2f}"])
stats_rows.append(["unique_sites", stats['unique_stations']])
stats_rows.append(["total_site_species_combinations", stats['total_combinations']])
stats_rows.append(["unique_species", stats['unique_pollutants']])
stats_rows.append(["unique_locations", stats['unique_locations']])
stats_rows.append(["expected_site_species_pairs", stats.get('expected_pairs', 0)])
stats_rows.append(["collected_site_species_pairs", stats.get('collected_pairs', 0)])
stats_rows.append(["missing_site_species_pairs_count", stats.get('missing_pairs_count', 0)])
stats_rows.append(["extra_site_species_pairs_count", stats.get('extra_pairs_count', 0)])
stats_rows.append(["total_nan_replacements", stats['total_nan_replacements']])
stats_rows.append(["mean_invalid_pct", f"{stats['mean_invalid_percentage']:.2f}"])
stats_rows.append(["max_invalid_pct", f"{stats['max_invalid_percentage']:.2f}"])

# Add year-specific metrics
for year in ['2023', '2024', '2025']:
    stats_rows.append([f"files_{year}", stats['files_by_year'].get(year, 0)])
    stats_rows.append([f"records_{year}", stats['records_by_year'].get(year, 0)])
    stats_rows.append([f"missing_{year}", stats['missing_by_year'].get(year, 0)])
    year_key = f'{year}measurements'
    stats_rows.append([f"replacements_{year}", stats['nan_replacements_by_year'].get(year_key, 0)])

# Save to csv stats report
pd.DataFrame(stats_rows[1:], columns=stats_rows[0]).to_csv(stats_output_path, index=False)
print(f"\nStatistics saved to: {stats_output_path}")

# Save species (pollutant) distribution to csv
total_combinations = stats['total_combinations']
species_distribution_df = pd.DataFrame(
    [
        {
            'SpeciesCode': k,
            'count': v,
            'percentage': round((v / total_combinations) * 100, 2) if total_combinations > 0 else 0
        }
        for k, v in stats['pollutant_distribution'].items()
    ]
)
species_distribution_df.to_csv(pollutant_distrubution_path, index=False)
print(f"Species (pollutant) distribution saved to: {pollutant_distrubution_path}")

# Save missing value distribution by species to csv
if stats.get('missing_by_pollutant_type'):
    missing_by_species_df = pd.DataFrame([
        {
            'SpeciesCode': k,
            'total_records': v['total_records'],
            'total_missing': v['total_missing'],
            'percentage_missing': v['percentage_missing']
        }
        for k, v in stats['missing_by_pollutant_type'].items()
    ])
    missing_by_species_df.to_csv(nan_val_pollutant_split_path, index=False)
    print(f"Missing value distribution by species saved to: {nan_val_pollutant_split_path}")

# Save missing values by site/species to csv
if stats.get('missing_by_station_pollutant'):
    missing_by_site_species_df = pd.DataFrame([
        {
            'SiteCode': k[0],
            'SpeciesCode': k[1],
            'missing': v[0],
            'total_row': v[1],
            'percentage_missing': (v[0] / v[1] * 100) if v[1] > 0 else 0
        }
        for k, v in stats['missing_by_station_pollutant'].items()
    ])
    missing_by_site_species_df.to_csv(nan_val_stationPollutant_path, index=False)
    print(f"Missing values by site/species saved to: {nan_val_stationPollutant_path}")


loading metadata...
  expected SiteCode/SpeciesCode pairs from metadata: 170

scanning optimased directory for collected data...

Total CSV files found: 4932

Reading all CSV files to calculate statistics...
  2023_mar: 141 files, 101,520 records, 14,883 missing (14.66%)
  2025_feb: 141 files, 91,368 records, 12,034 missing (13.17%)
  2024_feb: 141 files, 94,752 records, 9,582 missing (10.11%)
  2025_aug: 141 files, 101,520 records, 16,123 missing (15.88%)
  2024_aug: 141 files, 101,520 records, 19,425 missing (19.13%)
  2025_mar: 141 files, 101,520 records, 15,384 missing (15.15%)
  2023_feb: 141 files, 91,368 records, 12,838 missing (14.05%)
  2024_mar: 141 files, 101,520 records, 11,279 missing (11.11%)
  2023_aug: 141 files, 101,520 records, 11,360 missing (11.19%)
  2024_jul: 141 files, 101,520 records, 12,934 missing (12.74%)
  2025_jul: 141 files, 101,520 records, 16,506 missing (16.26%)
  2024_oct: 141 files, 101,520 records, 11,079 missing (10.91%)
  2023_sep: 141 files, 98,1

    loading metadata...
    expected SiteCode/SpeciesCode pairs from metadata: 170

    scanning optimased directory for collected data...

    Total CSV files found: 4932

    Reading all CSV files to calculate statistics...
    2023_mar: 141 files, 101,520 records, 14,883 missing (14.66%)
    2025_feb: 141 files, 91,368 records, 12,034 missing (13.17%)
    2024_feb: 141 files, 94,752 records, 9,582 missing (10.11%)
    2025_aug: 141 files, 101,520 records, 16,123 missing (15.88%)
    2024_aug: 141 files, 101,520 records, 19,425 missing (19.13%)
    2025_mar: 141 files, 101,520 records, 15,384 missing (15.15%)
    2023_feb: 141 files, 91,368 records, 12,838 missing (14.05%)
    2024_mar: 141 files, 101,520 records, 11,279 missing (11.11%)
    2023_aug: 141 files, 101,520 records, 11,360 missing (11.19%)
    2024_jul: 141 files, 101,520 records, 12,934 missing (12.74%)
    2025_jul: 141 files, 101,520 records, 16,506 missing (16.26%)
    2024_oct: 141 files, 101,520 records, 11,079 missing (10.91%)
    2023_sep: 141 files, 98,136 records, 11,727 missing (11.95%)
    2025_oct: 141 files, 101,520 records, 12,342 missing (12.16%)
    2023_jan: 141 files, 101,520 records, 17,911 missing (17.64%)
    2023_jul: 141 files, 101,520 records, 11,160 missing (10.99%)
    2024_jan: 141 files, 101,520 records, 10,375 missing (10.22%)
    2025_sep: 141 files, 98,136 records, 18,344 missing (18.69%)
    2024_sep: 141 files, 98,136 records, 15,469 missing (15.76%)
    2023_oct: 141 files, 101,520 records, 13,964 missing (13.75%)
    2025_jan: 141 files, 101,520 records, 11,198 missing (11.03%)
    2024_dec: 141 files, 101,520 records, 9,421 missing (9.28%)
    2024_apr: 141 files, 98,136 records, 11,208 missing (11.42%)
    2024_nov: 141 files, 98,136 records, 9,063 missing (9.24%)
    2023_may: 141 files, 101,520 records, 12,641 missing (12.45%)
    2025_nov: 141 files, 60,912 records, 8,613 missing (14.14%)
    2025_apr: 141 files, 98,136 records, 10,606 missing (10.81%)
    2024_may: 141 files, 101,520 records, 12,839 missing (12.65%)
    2025_may: 141 files, 101,520 records, 10,843 missing (10.68%)
    2023_nov: 138 files, 96,048 records, 10,851 missing (11.30%)
    2023_apr: 141 files, 98,136 records, 10,992 missing (11.20%)
    2023_dec: 141 files, 101,520 records, 14,882 missing (14.66%)
    2025_jun: 141 files, 98,136 records, 14,018 missing (14.28%)
    2024_jun: 141 files, 98,136 records, 9,661 missing (9.84%)
    2023_jun: 141 files, 98,136 records, 12,103 missing (12.33%)

    cross-referencing collected data with metadata...
    expected pairs from metadata: 170
    actually collected pairs: 141
    missing pairs (in metadata but not collected): 53
    extra pairs (collected but not in metadata): 24

    ========================================
    LAQN dataset statistics: initial assessment
    ========================================

    Scale and scope:
    Total files collected: 4,932
    Total measurement records: 3,446,208
    Total missing values (@Value): 443,658
    Overall completeness: 87.13%
    Unique monitoring sites (SiteCode): 78
    Total site-species combinations: 173
    Unique pollutant types (SpeciesCode): 6
    Unique geographic locations: 76

    Data collection coverage:
    Expected SiteCode/SpeciesCode pairs (from metadata): 170
    Actually collected pairs: 141
    Missing pairs (not collected): 53
    Extra pairs (not in metadata): 24

    Warning: 53 SiteCode/SpeciesCode pairs from metadata were not found in collected data.
    First 10 missing pairs:
    1. BL0 - PM25
    2. TH4 - PM25
    3. BT6 - PM25
    4. MEB - PM25
    5. GN6 - PM25
    6. GR8 - PM25
    7. GN3 - PM25
    8. TL6 - PM25
    9. GT1 - PM25
    10. CE3 - PM25

    Note: 24 SiteCode/SpeciesCode pairs in collected data are not in metadata.

    Files by year:
    2023: 1,689 files
    2025: 1,551 files
    2024: 1,692 files

    Records by year:
    2023: 1,192,464 records, 155,312 missing (13.02%)
    2025: 1,055,808 records, 146,011 missing (13.83%)
    2024: 1,197,936 records, 142,335 missing (11.88%)

    NaN replacement summary:
    Total invalid flags replaced: 0
    Mean invalid percentage per file: 0.00%
    Max invalid percentage: 0.00%

    Temporal coverage:
    Start date: 2023-01-01
    End date: 2025-11-19
    Total months: 35

    Pollutant (SpeciesCode) distribution:
    Site/species combinations by type:
    NO2: 60 (34.7%)
    PM25: 53 (30.6%)
    PM10: 43 (24.9%)
    O3: 11 (6.4%)
    SO2: 4 (2.3%)
    CO: 2 (1.2%)

    Missing value distribution by pollutant type (SpeciesCode):
    SpeciesCode            total records      missing    % missing
    ------------------------------------------------------------
    O3                           268,320       47,056       17.54%
    PM2.5                        586,944      100,755       17.17%
    SO2                           97,824       15,803       16.15%
    PM10                       1,026,456      126,749       12.35%
    NO2                        1,417,752      148,803       10.50%
    CO                            48,912        4,492        9.18%

    Missing values by site/species (SiteCode/SpeciesCode):
    SiteCode             SpeciesCode             missing    total_row    % missing
    ------------------------------------------------------------
    WM6                  PM10                     15,357       24,456       62.79%
    CE3                  NO2                      11,394       24,456       46.59%
    TL4                  NO2                       9,869       24,456       40.35%
    RI2                  O3                        9,732       24,456       39.79%
    WA7                  NO2                       9,236       24,456       37.77%
    BG1                  SO2                       8,373       24,456       34.24%
    TH4                  PM2.5                     8,042       24,456       32.88%
    CD1                  PM10                      7,711       24,456       31.53%
    WAA                  NO2                       7,697       24,456       31.47%
    CE3                  PM10                      7,652       24,456       31.29%
    CE3                  PM2.5                     7,652       24,456       31.29%
    TH4                  PM10                      7,457       24,456       30.49%
    CD1                  PM2.5                     7,339       24,456       30.01%
    GN6                  PM2.5                     7,240       24,456       29.60%
    CR8                  PM2.5                     7,207       24,456       29.47%
    GN0                  PM2.5                     7,099       24,456       29.03%
    HG4                  O3                        7,092       24,456       29.00%
    CD1                  NO2                       6,984       24,456       28.56%
    BT5                  PM2.5                     6,783       24,456       27.74%
    MY1                  O3                        6,634       24,456       27.13%

    Statistics saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/laqn_stats.csv
    Species (pollutant) distribution saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/pollutant_distribution.csv
    Missing value distribution by species saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/nan_values_by_pollutant.csv
    Missing values by site/species saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/nan_values_by_station_pollutant.csv

## 3) Data Quality validations:


A critical gap from the laqn report by applying formal statistical tests to validate data quality patterns. While descriptive statistics show 0% (before I notice the flags of the dataset) issue rate, I need statistical evidence that this pattern is real and not due to chance.


#### Purpuse:
 Checking data qualities if it is in the limits of eea, and make sence for general logic.
- Outlier detection in pollutant measurements.
- Data validity ranges based on WHO/EEA standards.
- Measurement consistency across time periods.
- Quality flags and suspicious patterns.

### methodology
 applies environmental data quality assessment standards:
1. Load aggregated measurement data from all csv files.
2. Calculate statistical distributions for each pollutant type.
3. Identify outliers using IQR method and domain knowledge.
4. Check values against established valid ranges.
5. Flag suspicious patterns constant values, extreme spikes.
6. Calculate quality scores for each station-pollutant combination.

#### air quality measurement standards

- Uk air quality objectives, limits and policy.
- https://uk-air.defra.gov.uk/air-pollution/uk-limits
- chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://uk-air.defra.gov.uk/assets/documents/Air_Quality_Objectives_Update_20230403.pdf

- DEFRA. (2023). *Air Pollution in the UK 2022*.
  - Source: https://uk-air.defra.gov.uk/library/annualreport/
  - Air Quality Objectives and limit values
  - Compliance assessment methodology

- UK Air Information Resource. (2024). *Air Pollution: UK Limits*.
  - Source: https://uk-air.defra.gov.uk/air-pollution/uk-limits
  - Current UK air quality objectives
  - Legal limit values and target dates
  - Measurement unit specifications (µg/m³)

  -  for the rest of the pollutants


- uk voc policy:
  - https://assets.publishing.service.gov.uk/media/5d7a2912ed915d522e4164a5/VO__statement_Final_12092019_CS__1_.pdf



In [ ]:

def find_negative_values(base_dir, dry_run=True):
    summary = []
    all_csvs = list(Path(base_dir).rglob("*.csv"))
    for csv_file in all_csvs:
        try:
            df = pd.read_csv(csv_file)
            if '@Value' in df.columns and 'SpeciesCode' in df.columns and 'SiteCode' in df.columns:
                df_valid = df[pd.to_numeric(df['@Value'], errors='coerce').notnull()].copy()
                df_valid['@Value'] = df_valid['@Value'].astype(float)
                negatives = df_valid[df_valid['@Value'] < 0]
                if not negatives.empty:
                    grouped = negatives.groupby(['SpeciesCode', 'SiteCode']).size().reset_index(name='neg_count')
                    for _, row in grouped.iterrows():
                        total = df_valid[(df_valid['SpeciesCode'] == row['SpeciesCode']) & (df_valid['SiteCode'] == row['SiteCode'])].shape[0]
                        percent = (row['neg_count'] / total * 100) if total > 0 else 0
                        summary.append({
                            'SpeciesCode': row['SpeciesCode'],
                            'SiteCode': row['SiteCode'],
                            'NegativeCount': row['neg_count'],
                            'TotalCount': total,
                            'PercentNegative': round(percent, 2),
                            'File': str(csv_file)
                        })
        except Exception as e:
            print(f"Warning: Could not process {csv_file}: {e}")

    summary_df = pd.DataFrame(summary)
    if not summary_df.empty:
        print("Negative value summary (by pollutant and site):")
        display(summary_df)
    else:
        print("No negative values found in the scanned files.")
    return summary_df

In [44]:
#  Example usage:
neg_summary = find_negative_values(base_dir)
neg_summary.to_csv("laqn_negative_value_summary.csv", index=False)

No negative values found in the scanned files.


In [41]:
def replace_negatives_with_nan(base_dir):
    base_dir = Path(base_dir)
    for csv_file in base_dir.rglob('*.csv'):
        df = pd.read_csv(csv_file)
        if '@Value' in df.columns:
            df['@Value'] = pd.to_numeric(df['@Value'], errors='coerce')
            df.loc[df['@Value'] < 0, '@Value'] = np.nan
            df.to_csv(csv_file, index=False)
        else:
            print(f"Skipped (no @Value column): {csv_file}")


In [42]:
# Example usage:
replace_negatives_with_nan(base_dir)

In [65]:
def calculate_quality_metrics(base_dir, csv_output_path):
    """
    validates laqn measurements against uk air quality objectives with averaging periods.
    
    - path: data/laqn/optimased/YYYY_month/SiteCode_SpeciesCode_YYYY-MM-DD_YYYY-MM-DD.csv
    - columns: @MeasurementDateGMT, @Value, SpeciesCode, SiteCode, SpeciesName, SiteName, SiteType, Latitude, Longitude
    
    parameters:
        base_dir : path to laqn optimased directory (e.g., data/laqn/optimased/)
        uk_limits_path : path to uk_pollutant_limits.csv from parsed pdf
    
    """
    if not Path(csv_output_path).exists():
        print(f"error: uk limits file not found at {csv_output_path}")
        return {}
    
    # load uk legal limits from parsed pdf
    uk_limits = pd.read_csv(csv_output_path, encoding="utf-8")
    uk_limits.columns = [col.strip().replace(' ', '_') for col in uk_limits.columns]
    uk_limits_dict = {}
    
    for _, row in uk_limits.iterrows():
        poll_std = row['pollutant_std']
        limit_val = row['limit']
        conc_type = str(row['concentration_measured_as']).lower().strip()
        unit = row['unit']
        
        if pd.notna(poll_std) and pd.notna(limit_val):
            if poll_std not in uk_limits_dict:
                uk_limits_dict[poll_std] = []
            
            # averaging period detection
            avg_period = 'unknown'
            if 'annual' in conc_type and 'running' in conc_type:
                avg_period = 'running_annual'
            elif 'running annual' in conc_type:
                avg_period = 'running_annual'
            elif 'annual' in conc_type:
                avg_period = 'annual'
            elif '24 hour' in conc_type or '24-hour' in conc_type:
                avg_period = '24hour'
            elif '8 hour' in conc_type or '8-hour' in conc_type:
                avg_period = '8hour'
            elif '1 hour' in conc_type or '1-hour' in conc_type or 'hour mean' in conc_type:
                avg_period = '1hour'
            elif 'maximum daily' in conc_type:
                avg_period = 'daily_max'
            
            uk_limits_dict[poll_std].append({
                'limit': float(limit_val),
                'type': conc_type,
                'unit': unit,
                'avg_period': avg_period
            })
    
    print(f"\nuk limits loaded for {len(uk_limits_dict)} pollutants:")
    for poll, limits in uk_limits_dict.items():
        period_info = ', '.join([f"{lim['avg_period']}: {lim['limit']}" for lim in limits])
        print(f"  {poll}: {period_info}")
    
    # load all laqn measurement data with timestamp
    print("\nloading laqn measurement data...")
    all_data = []
    base_path = Path(base_dir)
    
    # iterate through year-month folders (e.g., 2023_jan, 2024_feb, etc.)
    for year_month_dir in base_path.glob('*'):
        if year_month_dir.is_dir():
            print(f"processing {year_month_dir.name}...")
            # iterate through csv files (SiteCode_SpeciesCode_YYYY-MM-DD_YYYY-MM-DD.csv)
            for csv_file in year_month_dir.glob('*.csv'):
                try:
                    df = pd.read_csv(csv_file)
                    # check for required columns
                    required_cols = {'@MeasurementDateGMT', '@Value', 'SpeciesCode'}
                    if not required_cols.issubset(df.columns):
                        print(f"skipped {csv_file}: missing required columns. found: {list(df.columns)}")
                        continue
                    
                    if not df.empty:
                        all_data.append(df)
                except Exception as e:
                    print(f"error reading {csv_file}: {e}")
    
    if not all_data:
        print("error: no measurement data found")
        return {}
    
    df_all = pd.concat(all_data, ignore_index=True)
    print(f"loaded {len(df_all):,} total records")
    
    # filter valid values and parse timestamp
    df_all['@Value'] = pd.to_numeric(df_all['@Value'], errors='coerce')
    df_valid = df_all[df_all['@Value'].notna()].copy()
    
    # parse timestamp - laqn uses ISO format with timezone
    df_valid['@MeasurementDateGMT'] = pd.to_datetime(df_valid['@MeasurementDateGMT'], errors='coerce')
    df_valid = df_valid[df_valid['@MeasurementDateGMT'].notna()]
    
    print(f"analysing {len(df_valid):,} valid measurements with timestamps")
    
    # calculate quality metrics for each pollutant
    print("\nprocessing quality metrics by pollutant...")
    quality_results = {}
    
    for pollutant in df_valid['SpeciesCode'].unique():
        if pd.isna(pollutant):
            continue
        
        print(f"\nprocessing {pollutant}...")
        
        poll_data = df_valid[df_valid['SpeciesCode'] == pollutant].copy()
        
        if len(poll_data) == 0:
            continue
        
        # basic statistics on raw hourly data
        q_metrics = {
            'pollutant': pollutant,
            'count': int(len(poll_data)),
            'mean_hourly': float(poll_data['@Value'].mean()),
            'median_hourly': float(poll_data['@Value'].median()),
            'std_hourly': float(poll_data['@Value'].std()),
            'min': float(poll_data['@Value'].min()),
            'max': float(poll_data['@Value'].max()),
            'p95': float(poll_data['@Value'].quantile(0.95)),
            'p99': float(poll_data['@Value'].quantile(0.99))
        }
        
        # check for suspicious values
        negative_count = (poll_data['@Value'] < 0).sum()
        zero_count = (poll_data['@Value'] == 0).sum()
        
        q_metrics['negative_values'] = int(negative_count)
        q_metrics['negative_pct'] = float((negative_count / len(poll_data) * 100))
        q_metrics['zero_values'] = int(zero_count)
        q_metrics['zero_pct'] = float((zero_count / len(poll_data) * 100))
        
        # laqn uses standard codes (NO2, PM10, PM25, SO2, CO, O3) - direct match to uk limits
        # but PM2.5 might appear as PM25 in laqn
        poll_std_code = pollutant
        if pollutant == 'PM25':
            poll_std_code = 'PM25'  # uk limits use PM25
        
        # check against uk limits with proper averaging
        if poll_std_code in uk_limits_dict:
            uk_poll_limits = uk_limits_dict[poll_std_code]
            
            for limit_info in uk_poll_limits:
                avg_period = limit_info['avg_period']
                limit_value = limit_info['limit']
                
                if avg_period == 'annual':
                    poll_data['year'] = poll_data['@MeasurementDateGMT'].dt.year
                    annual_means = poll_data.groupby('year')['@Value'].mean()
                    
                    q_metrics['uk_annual_limit'] = limit_value
                    q_metrics['mean_annual'] = float(annual_means.mean())
                    q_metrics['exceeds_uk_annual'] = q_metrics['mean_annual'] > limit_value
                    
                    print(f"  annual mean: {q_metrics['mean_annual']:.2f} vs limit {limit_value}")
                
                elif avg_period == '24hour':
                    poll_data['date'] = poll_data['@MeasurementDateGMT'].dt.date
                    daily_means = poll_data.groupby('date')['@Value'].mean()
                    
                    exceedances = (daily_means > limit_value).sum()
                    
                    q_metrics['uk_24hour_limit'] = limit_value
                    q_metrics['daily_exceedances'] = int(exceedances)
                    q_metrics['daily_exceedances_pct'] = float((exceedances / len(daily_means) * 100))
                    
                    print(f"  24-hour: {exceedances} days exceed {limit_value}")
                
                elif avg_period == '8hour':
                    poll_data_sorted = poll_data.sort_values('@MeasurementDateGMT')
                    poll_data_sorted['rolling_8h'] = poll_data_sorted['@Value'].rolling(window=8, min_periods=6).mean()
                    
                    exceedances = (poll_data_sorted['rolling_8h'] > limit_value).sum()
                    
                    q_metrics['uk_8hour_limit'] = limit_value
                    q_metrics['8hour_exceedances'] = int(exceedances)
                    q_metrics['8hour_exceedances_pct'] = float((exceedances / len(poll_data_sorted) * 100))
                    
                    print(f"  8-hour: {exceedances} periods exceed {limit_value}")
                
                elif avg_period == '1hour':
                    exceedances = (poll_data['@Value'] > limit_value).sum()
                    
                    q_metrics['uk_1hour_limit'] = limit_value
                    q_metrics['hourly_exceedances'] = int(exceedances)
                    q_metrics['hourly_exceedances_pct'] = float((exceedances / len(poll_data) * 100))
                    
                    print(f"  1-hour: {exceedances} hours exceed {limit_value}")
                
                elif avg_period == 'running_annual':
                    poll_data_sorted = poll_data.sort_values('@MeasurementDateGMT')
                    poll_data_sorted['rolling_annual'] = poll_data_sorted['@Value'].rolling(window=24*365, min_periods=24*300).mean()
                    
                    q_metrics['uk_running_annual_limit'] = limit_value
                    q_metrics['mean_running_annual'] = float(poll_data_sorted['rolling_annual'].mean())
                    q_metrics['exceeds_running_annual'] = q_metrics['mean_running_annual'] > limit_value
                    
                    print(f"  running annual: {q_metrics['mean_running_annual']:.2f} vs limit {limit_value}")
                
                elif avg_period == 'daily_max':
                    poll_data_sorted = poll_data.sort_values('@MeasurementDateGMT')
                    poll_data_sorted['date'] = poll_data_sorted['@MeasurementDateGMT'].dt.date
                    poll_data_sorted['rolling_8h'] = poll_data_sorted['@Value'].rolling(window=8, min_periods=6).mean()
                    
                    daily_max = poll_data_sorted.groupby('date')['rolling_8h'].max()
                    exceedances = (daily_max > limit_value).sum()
                    
                    q_metrics['uk_daily_max_limit'] = limit_value
                    q_metrics['daily_max_exceedances'] = int(exceedances)
                    
                    print(f"  daily max 8h: {exceedances} days exceed {limit_value}")
            
            # overall assessment: use most restrictive limit for out of range check
            all_limits = [lim['limit'] for lim in uk_poll_limits]
            max_limit = max(all_limits)
            
            # define extreme threshold as 10x highest uk limit
            extreme_threshold = max_limit * 10
            out_of_range = (poll_data['@Value'] > extreme_threshold).sum()
            
            q_metrics['extreme_threshold'] = extreme_threshold
            q_metrics['out_of_range'] = int(out_of_range)
            q_metrics['out_of_range_pct'] = float((out_of_range / len(poll_data) * 100))
        
        else:
            print(f"  no uk limits defined for {poll_std_code}")
            q_metrics['uk_annual_limit'] = None
            q_metrics['exceeds_uk_annual'] = False
            q_metrics['out_of_range'] = 0
            q_metrics['out_of_range_pct'] = 0.0
        
        # always calculate annual mean
        poll_data['year'] = poll_data['@MeasurementDateGMT'].dt.year
        annual_means = poll_data.groupby('year')['@Value'].mean()
        q_metrics['mean_annual'] = float(annual_means.mean())
        
        # if annual limit exists, compare
        if 'uk_annual_limit' in q_metrics and q_metrics['uk_annual_limit'] is not None:
            q_metrics['exceeds_uk_annual'] = q_metrics['mean_annual'] > q_metrics['uk_annual_limit']
        else:
            q_metrics['exceeds_uk_annual'] = None
        
        # ozone (O3): count days where max running 8-hour mean > 100 µg/m³
        if poll_std_code == 'O3':
            poll_data_sorted = poll_data.sort_values('@MeasurementDateGMT')
            poll_data_sorted['rolling_8h'] = poll_data_sorted['@Value'].rolling(window=8, min_periods=6).mean()
            poll_data_sorted['date'] = poll_data_sorted['@MeasurementDateGMT'].dt.date
            daily_max_8h = poll_data_sorted.groupby('date')['rolling_8h'].max()
            o3_exceedance_days = (daily_max_8h > 100).sum()
            q_metrics['o3_exceedance_days'] = int(o3_exceedance_days)
        
        # CO: maximum daily running 8-hour mean
        if poll_std_code == 'CO':
            poll_data_sorted = poll_data.sort_values('@MeasurementDateGMT')
            poll_data_sorted['date'] = poll_data_sorted['@MeasurementDateGMT'].dt.date
            poll_data_sorted['rolling_8h'] = poll_data_sorted['@Value'].rolling(window=8, min_periods=6).mean()
            daily_max_8h = poll_data_sorted.groupby('date')['rolling_8h'].max()
            co_max_daily_8h_mean = daily_max_8h.max()
            q_metrics['co_max_daily_8h_mean'] = float(co_max_daily_8h_mean)
        
        quality_results[pollutant] = q_metrics
    
    return quality_results

In [66]:
def print_quality_metrics(quality_results):
    """
    print comprehensive quality metrics report with uk compliance for laqn data.
    
    parameters:
        quality_results : dict
            dictionary returned by calculate_quality_metrics_laqn
    """
    
    print("\n" + "="*40)
    print("laqn quality metrics report")
    print("="*40)
    
    for poll, metrics in quality_results.items():
        print(f"\n{poll}:")
        print(f"  total measurements: {metrics['count']:,}")
        print(f"  hourly mean: {metrics['mean_hourly']:.2f}")
        
        if 'mean_annual' in metrics:
            print(f"  annual mean: {metrics['mean_annual']:.2f}", end="")
            if 'uk_annual_limit' in metrics and metrics['uk_annual_limit'] is not None:
                print(f" (limit: {metrics['uk_annual_limit']})")
                status = "exceeds" if metrics['exceeds_uk_annual'] else "compliant"
                print(f"    status: {status}")
            else:
                print(" (no uk annual limit)")
        
        if 'o3_exceedance_days' in metrics:
            print(f"  O3 8-hour mean exceedance days: {metrics['o3_exceedance_days']}")
        if 'co_max_daily_8h_mean' in metrics:
            print(f"  CO max daily 8-hour mean: {metrics['co_max_daily_8h_mean']:.2f}")
        
        if 'daily_exceedances' in metrics:
            print(f"  24-hour exceedances: {metrics['daily_exceedances']} days")
        if 'hourly_exceedances' in metrics:
            print(f"  1-hour exceedances: {metrics['hourly_exceedances']} hours")
        if metrics['negative_values'] > 0:
            print(f"  warning: {metrics['negative_values']} negative values")
        if metrics['out_of_range'] > 0:
            print(f"  warning: {metrics['out_of_range']} extreme values")
    
    print("="*40)



    

In [68]:
# calculate quality metrics
quality_results = calculate_quality_metrics(base_dir, csv_output_path)

print_quality_metrics(quality_results)

if quality_results:
    # save comprehensive report
    print("\nsaving laqn quality metrics report...")
    
    quality_rows = []
    for poll, metrics in quality_results.items():
        row = {
            'pollutant': metrics['pollutant'],
            'total_measurements': metrics['count'],
            'mean_hourly': f"{metrics['mean_hourly']:.2f}",
            'min': f"{metrics['min']:.2f}",
            'max': f"{metrics['max']:.2f}",
            'p95': f"{metrics['p95']:.2f}",
            'negative_values': metrics['negative_values'],
            'zero_values': metrics['zero_values'],
            'out_of_range': metrics['out_of_range']
        }
        
        # add uk limit compliance fields
        if 'uk_annual_limit' in metrics and metrics['uk_annual_limit']:
            row['uk_annual_limit'] = metrics['uk_annual_limit']
            row['mean_annual'] = f"{metrics['mean_annual']:.2f}" if 'mean_annual' in metrics else 'n/a'
            row['exceeds_annual'] = 'yes' if metrics.get('exceeds_uk_annual', False) else 'no'
        
        if 'daily_exceedances' in metrics:
            row['uk_24hour_limit'] = metrics['uk_24hour_limit']
            row['daily_exceedances'] = metrics['daily_exceedances']
        
        if 'hourly_exceedances' in metrics:
            row['uk_1hour_limit'] = metrics['uk_1hour_limit']
            row['hourly_exceedances'] = metrics['hourly_exceedances']
        
        if 'o3_exceedance_days' in metrics:
            row['o3_exceedance_days'] = metrics['o3_exceedance_days']
        
        if 'co_max_daily_8h_mean' in metrics:
            row['co_max_daily_8h_mean'] = f"{metrics['co_max_daily_8h_mean']:.2f}"
        
        quality_rows.append(row)
    
    #ssave to quality metrics csv
    pd.DataFrame(quality_rows).to_csv("quality_metrics.csv", index=False)
    print(f"saved to: {quality_results}")
    print("done")
else:
    print("laqn quality metrics calculation failed")


uk limits loaded for 11 pollutants:
  PM10: 24hour: 50.0, annual: 40.0
  PM2.5: annual: 20.0
  NO2: annual: 40.0
  O3: 8hour: 100.0
  SO2: 24hour: 125.0
  PAH: annual: 0.25
  Benzene: running_annual: 16.25
  1,3-butadiene: running_annual: 2.25
  CO: daily_max: 10.0
  LEAD: annual: 0.5
  NOx: annual: 30.0

loading laqn measurement data...
processing 2023_mar...
processing 2025_feb...
processing 2024_feb...
processing 2025_aug...
processing 2024_aug...
processing 2025_mar...
processing 2023_feb...
processing 2024_mar...
processing 2023_aug...
processing 2024_jul...
processing 2025_jul...
processing 2024_oct...
processing 2023_sep...
processing 2025_oct...
processing 2023_jan...
processing 2023_jul...
processing 2024_jan...
processing 2025_sep...
processing 2024_sep...
processing 2023_oct...
processing 2025_jan...
processing 2024_dec...
processing 2024_apr...
processing 2024_nov...
processing 2023_may...
processing 2025_nov...
processing 2025_apr...
processing 2024_may...
processing 2025

    uk limits loaded for 11 pollutants:
    PM10: 24hour: 50.0, annual: 40.0
    PM2.5: annual: 20.0
    NO2: annual: 40.0
    O3: 8hour: 100.0
    SO2: 24hour: 125.0
    PAH: annual: 0.25
    Benzene: running_annual: 16.25
    1,3-butadiene: running_annual: 2.25
    CO: daily_max: 10.0
    LEAD: annual: 0.5
    NOx: annual: 30.0

    loading laqn measurement data...
    processing 2023_mar...
    processing 2025_feb...
    processing 2024_feb...
    processing 2025_aug...
    processing 2024_aug...
    processing 2025_mar...
    processing 2023_feb...
    processing 2024_mar...
    processing 2023_aug...
    processing 2024_jul...
    processing 2025_jul...
    processing 2024_oct...
    processing 2023_sep...
    processing 2025_oct...
    processing 2023_jan...
    processing 2023_jul...
    processing 2024_jan...
    processing 2025_sep...
    processing 2024_sep...
    processing 2023_oct...
    processing 2025_jan...
    processing 2024_dec...
    processing 2024_apr...
    processing 2024_nov...
    processing 2023_may...
    processing 2025_nov...
    processing 2025_apr...
    processing 2024_may...
    processing 2025_may...
    processing 2023_nov...
    processing 2023_apr...
    processing 2023_dec...
    processing report...
    processing 2025_jun...
    processing 2024_jun...
    processing 2023_jun...
    loaded 3,446,208 total records
    analysing 2,981,417 valid measurements with timestamps

    processing quality metrics by pollutant...

    processing NO2...
    annual mean: 23.30 vs limit 40.0

    processing PM2.5...
    annual mean: 9.29 vs limit 20.0

    processing PM10...
    24-hour: 3 days exceed 50.0
    annual mean: 17.32 vs limit 40.0

    processing SO2...
    24-hour: 0 days exceed 125.0

    processing O3...
    8-hour: 2741 periods exceed 100.0

    processing CO...
    daily max 8h: 0 days exceed 10.0

    ========================================
    laqn quality metrics report
    ========================================

    NO2:
    total measurements: 1,267,018
    hourly mean: 23.34
    annual mean: 23.30 (limit: 40.0)
        status: compliant

    PM2.5:
    total measurements: 482,559
    hourly mean: 9.25
    annual mean: 9.29 (limit: 20.0)
        status: compliant
    warning: 11 extreme values

    PM10:
    total measurements: 896,146
    hourly mean: 17.23
    annual mean: 17.32 (limit: 40.0)
        status: compliant
    24-hour exceedances: 3 days
    warning: 3 extreme values

    SO2:
    total measurements: 71,217
    hourly mean: 1.47
    annual mean: 1.52 (no uk annual limit)
    24-hour exceedances: 0 days

    O3:
    total measurements: 220,912
    hourly mean: 47.71
    annual mean: 47.80 (no uk annual limit)
    O3 8-hour mean exceedance days: 70

    CO:
    total measurements: 43,565
    hourly mean: 0.20
    annual mean: 0.20 (no uk annual limit)
    CO max daily 8-hour mean: 2.14
    ========================================

    saving laqn quality metrics report...
    saved to: {'NO2': {'pollutant': 'NO2', 'count': 1267018, 'mean_hourly': 23.33811129755063, 'median_hourly': 19.0, 'std_hourly': 17.169706453593633, 'min': 0.0, 'max': 376.3, 'p95': 57.1, 'p99': 78.3, 'negative_values': 0, 'negative_pct': 0.0, 'zero_values': 632, 'zero_pct': 0.049880901455227944, 'uk_annual_limit': 40.0, 'mean_annual': 23.296558581709963, 'exceeds_uk_annual': False, 'extreme_threshold': 400.0, 'out_of_range': 0, 'out_of_range_pct': 0.0}, 'PM2.5': {'pollutant': 'PM2.5', 'count': 482559, 'mean_hourly': 9.249439550396948, 'median_hourly': 7.0, 'std_hourly': 7.869002875250606, 'min': 0.0, 'max': 909.0, 'p95': 23.9, 'p99': 38.8, 'negative_values': 0, 'negative_pct': 0.0, 'zero_values': 2090, 'zero_pct': 0.4331076614465796, 'uk_annual_limit': 20.0, 'mean_annual': 9.287265567932431, 'exceeds_uk_annual': False, 'extreme_threshold': 200.0, 'out_of_range': 11, 'out_of_range_pct': 0.0022795140076135767}, 'PM10': {'pollutant': 'PM10', 'count': 896146, 'mean_hourly': 17.23079754861373, 'median_hourly': 14.4, 'std_hourly': 12.470894582830518, 'min': 0.0, 'max': 759.0, 'p95': 39.0, 'p99': 61.8, 'negative_values': 0, 'negative_pct': 0.0, 'zero_values': 835, 'zero_pct': 0.09317678146194928, 'uk_24hour_limit': 50.0, 'daily_exceedances': 3, 'daily_exceedances_pct': 0.2944062806673209, 'uk_annual_limit': 40.0, 'mean_annual': 17.3172696741254, 'exceeds_uk_annual': False, 'extreme_threshold': 500.0, 'out_of_range': 3, 'out_of_range_pct': 0.0003347668795040094}, 'SO2': {'pollutant': 'SO2', 'count': 71217, 'mean_hourly': 1.46688431133016, 'median_hourly': 1.1, 'std_hourly': 3.2138526496443744, 'min': 0.0, 'max': 271.4, 'p95': 4.2, 'p99': 7.2, 'negative_values': 0, 'negative_pct': 0.0, 'zero_values': 2348, 'zero_pct': 3.296965612143168, 'uk_24hour_limit': 125.0, 'daily_exceedances': 0, 'daily_exceedances_pct': 0.0, 'extreme_threshold': 1250.0, 'out_of_range': 0, 'out_of_range_pct': 0.0, 'mean_annual': 1.5215136717118256, 'exceeds_uk_annual': None}, 'O3': {'pollutant': 'O3', 'count': 220912, 'mean_hourly': 47.70532836604621, 'median_hourly': 48.1, 'std_hourly': 23.953550068547706, 'min': 0.0, 'max': 198.6, 'p95': 85.6, 'p99': 108.68899999999849, 'negative_values': 0, 'negative_pct': 0.0, 'zero_values': 260, 'zero_pct': 0.11769392337220251, 'uk_8hour_limit': 100.0, '8hour_exceedances': 2741, '8hour_exceedances_pct': 1.2407655537046427, 'extreme_threshold': 1000.0, 'out_of_range': 0, 'out_of_range_pct': 0.0, 'mean_annual': 47.798197519325676, 'exceeds_uk_annual': None, 'o3_exceedance_days': 70}, 'CO': {'pollutant': 'CO', 'count': 43565, 'mean_hourly': 0.20249512223114888, 'median_hourly': 0.2, 'std_hourly': 0.18651998992979038, 'min': 0.0, 'max': 4.9, 'p95': 0.5, 'p99': 0.8, 'negative_values': 0, 'negative_pct': 0.0, 'zero_values': 6773, 'zero_pct': 15.546883966486858, 'uk_daily_max_limit': 10.0, 'daily_max_exceedances': 0, 'extreme_threshold': 100.0, 'out_of_range': 0, 'out_of_range_pct': 0.0, 'mean_annual': 0.2006708313657066, 'exceeds_uk_annual': None, 'co_max_daily_8h_mean': 2.1374999999999997}}
    done

## 5) Chi-Square Test for LAQN Data Quality

Uses statistical tests to mathematically prove that LAQN data collection process was consistent and reliable across time. 
It serves as a quality control check that ensures we didn't accidentally collect more data in some months than others, which could bias our analysis.

#### Why Chi-Square Test?
 - The chi-square test answers one simple question: Are my monthly file counts similar enough to trust, or are some months suspiciously different?
 - Air pollution varies by season
 - Policy decisions need unbiased evidence
 - Academic reviewers will question imbalanced datasets

### What Chi-Square Test Does

The chi-square test answers one simple question: Are my monthly file counts similar enough to trust, or are some months suspiciously different?

#### How It Works

1. What we observe: Count how many data files we have for each month.
2. What we expect: If data collection was perfect, each month should have roughly the same count.
3. The test: Measures how far observed counts are from the expected counts.
4. The result: Gives us a p-value that tells us if the differences are just random variation or a real problem.

In [70]:
def chi_square_tests(base_dir):
    """
    run statistical tests to prove laqn data collection was consistent.
    
    - chi-square test: checks if we have similar amounts of data for each year
    - if p-value < 0.05: data isn't evenly spread - potential problem
    - if p-value > 0.05: data is evenly spread - good
    
    parameters:
        base_dir : path to laqn optimised directory
            e.g., data/laqn/optimised/
            
    returns:
        dict with test results (chi2 statistic, p-value, counts)
    """
    
    # count files per year 2023:12 months, 2024:12 months, 2025:11 months up to nov
    yearly_data = {'2023': 0, '2024': 0, '2025': 0}
    year_months = {'2023': 12, '2024': 12, '2025': 11}
    
    base_path = Path(base_dir)
    
    # iterate through all month folders and count files by year
    for month_dir in base_path.glob('*'):
        if not month_dir.is_dir():
            continue
        
        # extract year from folder name (e.g., "2023_jan" ="2023")
        folder_name = month_dir.name
        try:
            year = folder_name.split('_')[0]
            if year in yearly_data:
                # count csv files in this month
                file_count = len(list(month_dir.glob('*.csv')))
                yearly_data[year] += file_count
        except Exception as e:
            print(f"warning: couldn't process {folder_name}: {e}")
            continue
    
    print("\nChi-square test for laqn yearly distribution")
    print("=" * 40)
    print("\nobserved file counts by year:")
    
    # prepare for chi-square
    year_counts = [yearly_data[y] for y in ['2023', '2024', '2025']]
    total_files = sum(year_counts)
    total_months = 35  # total months 35
    
    # Expected counts proportional to number of months
    expected_counts = [
        total_files * (year_months[year] / total_months)
        for year in ['2023', '2024', '2025']
    ]
    
    # display counts
    for year, count, expected in zip(['2023', '2024', '2025'], 
                                      year_counts, expected_counts):
        print(f"  {year}: {count:5d} files (expected: {expected:7.1f})")
    
    # run chi-square test
    chi2, p_value = stats.chisquare(
        f_obs=year_counts, 
        f_exp=expected_counts
    )
    
    print()
    print(f"chi-square statistic: {chi2:.4f}")
    print(f"p-value: {p_value:.4f}")
    print()
    
    # interpret results
    if p_value < 0.05:
        print(f"Resultreject null hypothesis (p < 0.05)")
        print(f"Interpretation: years not evenly distributed")
        print(f"Action: investigate why some years have different file counts")
    else:
        print(f"Result: accept null hypothesis (p >= 0.05)")
        print(f"Interpretation: years evenly distributed")
        print(f"Conclusion: data collection was consistent across years")
    
    print("=" * 40)
    
    return {
        'test': 'chi-square year-wise laqn',
        'chi2_statistic': chi2,
        'p_value': p_value,
        'year_counts': year_counts,
        'expected_counts': expected_counts,
        'total_files': total_files,
        'total_months': total_months
    }

In [71]:
# run chi-square test for laqn
print("Starting chi-square test for laqn data...")

test_results_laqn = chi_square_tests(base_dir)



pd.DataFrame([{
    'test_name': test_results_laqn['test'],
    'statistic': f"{test_results_laqn['chi2_statistic']:.4f}",
    'p_value': f"{test_results_laqn['p_value']:.4f}",
    'interpretation': ('evenly distributed' 
                      if test_results_laqn['p_value'] >= 0.05 
                      else 'unevenly distributed'),
    'total_files': test_results_laqn['total_files'],
    'total_months': test_results_laqn['total_months'],
    '2023_count': test_results_laqn['year_counts'][0],
    '2024_count': test_results_laqn['year_counts'][1],
    '2025_count': test_results_laqn['year_counts'][2],
    '2023_expected': f"{test_results_laqn['expected_counts'][0]:.1f}",
    '2024_expected': f"{test_results_laqn['expected_counts'][1]:.1f}",
    '2025_expected': f"{test_results_laqn['expected_counts'][2]:.1f}"
}]).to_csv(chi_square_laqn_output, index=False)

print(f"\nStatistical test results saved to: {chi_square_laqn_output}")
print("done")

Starting chi-square test for laqn data...

Chi-square test for laqn yearly distribution

observed file counts by year:
  2023:  1689 files (expected:  1691.0)
  2024:  1692 files (expected:  1691.0)
  2025:  1551 files (expected:  1550.1)

chi-square statistic: 0.0035
p-value: 0.9983

Result: accept null hypothesis (p >= 0.05)
Interpretation: years evenly distributed
Conclusion: data collection was consistent across years

Statistical test results saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/chi_square_tests.csv
done


    Starting chi-square test for laqn data...

    Chi-square test for laqn yearly distribution
    ========================================

    observed file counts by year:
    2023:  1689 files (expected:  1691.0)
    2024:  1692 files (expected:  1691.0)
    2025:  1551 files (expected:  1550.1)

    chi-square statistic: 0.0035
    p-value: 0.9983

    Result: accept null hypothesis (p >= 0.05)
    Interpretation: years evenly distributed
    Conclusion: data collection was consistent across years
    ========================================

    Statistical test results saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/chi_square_tests.csv
    done

### Year-over-Year Station Coverage Analysis

After confirming data distribution consistency with the chi-square test, we need to understand **what changed** between years. This analysis identifies:

- Which station-pollutant combinations stopped reporting
- Whether equipment was decommissioned or failed
- If monitoring network coverage changed over time

### Why This Matters

Even if file counts are statistically balanced, the **specific stations** might have changed:
- Equipment upgrades/replacements
- Station relocations
- Policy changes in monitoring priorities
- Budget constraints affecting network coverage

### What We Track

For each year (2023, 2024, 2025):
1. Count unique station-pollutant combinations
2. Identify combinations present in 2023 but missing in later years
3. Quantify network coverage changes

### Expected Findings

Based on LAQN data quality report:
- Some stations may have been decommissioned
- Equipment failures affecting specific pollutants
- Normal operational gaps vs. permanent losses

This analysis helps distinguish between:
- **Temporary gaps**: Equipment maintenance, calibration
- **Permanent losses**: Station closure, equipment decommissioning

In [74]:
def analyse_year_difference(base_dir):
    """
    find which station-pollutant combinations are missing in later years.
    
    analyses laqn network coverage changes by comparing active combinations
    across 2023, 2024, and 2025.
    
    parameters:
        base_dir 
    
    """
    
    base_path = Path(base_dir)
    
    # unique station-pollutant combinations per year
    year_files = {'2023': set(), '2024': set(), '2025': set()}
    
    # iterate through all month folders
    for month_dir in base_path.glob('*'):
        if not month_dir.is_dir():
            continue
        
        # extract year from folder name (e.g., "2023_jan" -> "2023")
        folder_name = month_dir.name
        try:
            year = folder_name.split('_')[0]
            if year not in year_files:
                continue
            
            # process all csv files in this month
            for csv_file in month_dir.glob('*.csv'):
                # file format: SiteCode_SpeciesCode_YYYY-MM-DD_YYYY-MM-DD.csv
                # example: BG1_NO2_2023-01-01_2023-01-31.csv
                filename = csv_file.stem
                parts = filename.split('_')
                
                if len(parts) >= 2:
                    site_code = parts[0]
                    species_code = parts[1]
                    combination = (site_code, species_code)
                    year_files[year].add(combination)
        
        except Exception as e:
            print(f"warning: couldn't process {folder_name}: {e}")
            continue
    
    # find what's in 2023 but missing in 2024/2025
    lost_2024 = year_files['2023'] - year_files['2024']
    lost_2025 = year_files['2023'] - year_files['2025']
    
    # also check what's new in later years
    new_2024 = year_files['2024'] - year_files['2023']
    new_2025 = year_files['2025'] - year_files['2023']
    
    print("\n" + "=" * 40)
    print("LAQN station/pollutant coverage analysis")
    print("=" * 40)
    
    print("\nyearly totals:")
    print(f"2023: {len(year_files['2023'])} unique station-pollutant combinations.")
    print(f"2024: {len(year_files['2024'])} unique station-pollutant combinations.")
    print(f"2025: {len(year_files['2025'])} unique station-pollutant combinations.")
    
    print("\nChanges over time:")
    print(f"Lost in 2024 (vs 2023): {len(lost_2024)} combinations")
    print(f"Lost in 2025 (vs 2023): {len(lost_2025)} combinations")
    print(f"New in 2024 (vs 2023): {len(new_2024)} combinations")
    print(f"New in 2025 (vs 2023): {len(new_2025)} combinations")
    
    # show examples of lost combinations
    if lost_2024:
        print("\nExamples of combinations lost in 2024:")
        for site, species in sorted(lost_2024)[:10]:
            print(f"    {site} - {species}")
        if len(lost_2024) > 10:
            print(f"    ... and {len(lost_2024) - 10} more")
    
    if lost_2025:
        print("\nExamples of combinations lost in 2025:")
        for site, species in sorted(lost_2025)[:10]:
            print(f"    {site} - {species}")
        if len(lost_2025) > 10:
            print(f"    ... and {len(lost_2025) - 10} more")
    
    # show examples of new combinations
    if new_2024:
        print("\nExamples of new combinations in 2024:")
        for site, species in sorted(new_2024)[:10]:
            print(f"    {site} - {species}")
        if len(new_2024) > 10:
            print(f"    ... and {len(new_2024) - 10} more")
    
    if new_2025:
        print("\nExamples of new combinations in 2025:")
        for site, species in sorted(new_2025)[:10]:
            print(f"    {site} - {species}")
        if len(new_2025) > 10:
            print(f"    ... and {len(new_2025) - 10} more")
    
    print("=" * 60)
    
    return {
        '2023_count': len(year_files['2023']),
        '2024_count': len(year_files['2024']),
        '2025_count': len(year_files['2025']),
        '2023_combinations': year_files['2023'],
        '2024_combinations': year_files['2024'],
        '2025_combinations': year_files['2025'],
        'lost_2024': lost_2024,
        'lost_2025': lost_2025,
        'new_2024': new_2024,
        'new_2025': new_2025
    }

In [75]:
# run year difference analysis for laqn
print("Analysing station-pollutant coverage changes across years...")

analysis_laqn = analyse_year_difference(base_dir)


# create summary dataframe
summary_data = {
    'year': ['2023', '2024', '2025'],
    'total_combinations': [
        analysis_laqn['2023_count'],
        analysis_laqn['2024_count'],
        analysis_laqn['2025_count']
    ],
    'lost_vs_2023': [0, len(analysis_laqn['lost_2024']), len(analysis_laqn['lost_2025'])],
    'new_vs_2023': [0, len(analysis_laqn['new_2024']), len(analysis_laqn['new_2025'])]
}

pd.DataFrame(summary_data).to_csv(year_diff_output, index=False)
print(f"\nyear coverage summary saved to: {year_diff_output}")

# optionally save detailed lists of lost/new combinations
if analysis_laqn['lost_2024'] or analysis_laqn['lost_2025']:
    lost_details_output = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "laqn" / "report" / "lost_combinations_detail.csv"
    
    lost_records = []
    
    for site, species in analysis_laqn['lost_2024']:
        lost_records.append({
            'site_code': site,
            'species_code': species,
            'year_lost': '2024',
            'present_in_2023': 'yes',
            'present_in_2024': 'no',
            'present_in_2025': 'yes' if (site, species) in analysis_laqn['2025_combinations'] else 'no'
        })
    
    for site, species in analysis_laqn['lost_2025']:
        if (site, species) not in analysis_laqn['lost_2024']:  # avoid duplicates
            lost_records.append({
                'site_code': site,
                'species_code': species,
                'year_lost': '2025',
                'present_in_2023': 'yes',
                'present_in_2024': 'yes' if (site, species) in analysis_laqn['2024_combinations'] else 'no',
                'present_in_2025': 'no'
            })
    
    if lost_records:
        pd.DataFrame(lost_records).to_csv(lost_details_output, index=False)
        print(f"Detailed lost combinations saved to: {lost_details_output}")

print("done")

Analysing station-pollutant coverage changes across years...

LAQN station/pollutant coverage analysis

yearly totals:
2023: 141 unique station-pollutant combinations.
2024: 141 unique station-pollutant combinations.
2025: 141 unique station-pollutant combinations.

Changes over time:
Lost in 2024 (vs 2023): 0 combinations
Lost in 2025 (vs 2023): 0 combinations
New in 2024 (vs 2023): 0 combinations
New in 2025 (vs 2023): 0 combinations

year coverage summary saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/year_coverage_analysis.csv
done


## 6) Seasonal Trend Analysis

Analyses the cleaned LAQN air quality data to identify:
- Temporal trends (yearly, monthly, daily patterns).
- Pollutant-specific characteristics.
- Geographic hotspots.
- Seasonal variations.
- Exceedances of UK legal limits.

### 1. Overall Pollutant Trends
 
- Yearly average comparison by pollutant.
- Monthly trends across all years.
- Statistical summary tables.
- Visualisations: bar charts & line plots.

### 2. Pollutant-Specific Analysis

- Time series plot (daily averages).
- Distribution histogram (mean, median).
- Hourly pattern (rush hour peaks).
- Monthly boxplots (seasonal variation).

### 3. Seasonal Patterns

- Winter/Spring/Summer/Autumn definitions.
- Seasonal averages per pollutant.
- Comparison visualisation.
- Geographic distribution (can add later).
 
### 4. UK Legal Limit Exceedances

- Compares actual measurements vs UK limits.
- Calculates exceedance percentages.
- Identifies which pollutants/stations exceed limits most.

##### Data Analysis and Results
##### Pollution patterns, trends, and insights from LAQN dataset (2023-2025)

This section analyses the cleaned LAQN air quality data to identify:
- Temporal trends (yearly, monthly, daily patterns).
- Pollutant-specific characteristics.
- Geographic hotspots.
- Seasonal variations.
- Exceedances of UK legal limits.

Structure:
1. Overall pollution trends (2023-2025).
2. Pollutant-specific analysis.
3. Temporal patterns (seasonal, weekly, daily).
4. Geographic distribution.
5. Limit exceedances analysis.
6. Key findings summary.




#### 1) Load pollution data.

Sample data for analysis. Begin with only analysing 100 files, that's why the run function is set up very bottom at 100.
Once quick exploration is done it will be changed.

In [76]:
def load_pollution_data(base_dir, sample_size=None, pollutants=None, stations=None):
    """
    Load a sample of data for analysis.

    For full dataset analysis, set sample_size=None. For quick exploration, use sample_size=100.

    Parameters:
        base_dir : path to laqn optimised directory
        sample_size : int, optional - number of files to load
        pollutants : list, optional - filter by species codes (e.g., ['NO2', 'PM25'])
        stations : list, optional - filter by site codes (e.g., ['BG1', 'MY1'])

    Returns:
        Combined data with cols: timestamp, value, station_name, pollutant_name, year, month, day.
    """
    base_dir = Path(base_dir)
    all_data = []
    file_count = 0

    # Iterate through month folders (2023_jan, 2024_feb, etc.)
    for month_dir in sorted(base_dir.glob('*')):
        if not month_dir.is_dir():
            continue
        
        # Get all CSV files in this month
        all_files = list(month_dir.glob('*.csv'))
        
        # Filter by pollutant if specified (files named: SiteCode_SpeciesCode_dates.csv)
        if pollutants:
            all_files = [f for f in all_files if any(f'_{p}_' in f.stem for p in pollutants)]
        
        # Filter by station if specified
        if stations:
            all_files = [f for f in all_files if any(f.stem.startswith(f'{s}_') for s in stations)]
        
        # Apply sample size limit
        if sample_size and file_count >= sample_size:
            break
        
        files_to_load = all_files
        if sample_size:
            remaining = sample_size - file_count
            files_to_load = all_files[:remaining]
        
        # Load each file
        for filepath in files_to_load:
            try:
                df = pd.read_csv(filepath)
                
                # Check required columns exist (LAQN format)
                if '@MeasurementDateGMT' not in df.columns or '@Value' not in df.columns:
                    continue
                
                # Standardise column names
                df = df.rename(columns={
                    '@MeasurementDateGMT': 'timestamp',
                    '@Value': 'value',
                    'SiteName': 'station_name',
                    'SpeciesCode': 'pollutant_name'
                })
                
                # Add metadata if missing
                if 'station_name' not in df.columns and 'SiteName' in df.columns:
                    df['station_name'] = df['SiteName']
                elif 'station_name' not in df.columns:
                    # Extract from filename: SiteCode_SpeciesCode_dates.csv
                    parts = filepath.stem.split('_')
                    df['station_name'] = parts[0] if len(parts) > 0 else 'unknown'
                
                if 'pollutant_name' not in df.columns:
                    parts = filepath.stem.split('_')
                    df['pollutant_name'] = parts[1] if len(parts) > 1 else 'unknown'
                
                all_data.append(df)
                file_count += 1
                
            except Exception as e:
                print(f"  Warning: couldn't load {filepath.name}: {e}")
                continue
    
    if not all_data:
        print("\n  Error: no data loaded.")
        return pd.DataFrame()
    
    # Combine all data
    data = pd.concat(all_data, ignore_index=True)
    
    # Parse timestamp and extract time components
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['year'] = data['timestamp'].dt.year
    data['month'] = data['timestamp'].dt.month
    data['day'] = data['timestamp'].dt.day
    data['hour'] = data['timestamp'].dt.hour
    data['dayofweek'] = data['timestamp'].dt.dayofweek
    data['week'] = data['timestamp'].dt.isocalendar().week
    
    # Add month name for plotting
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    data['month_name'] = data['month'].apply(lambda x: month_names[x-1])
    
    # Print summary
    print(f"\n  Loaded {len(data):,} measurements from {file_count} files.")
    print(f"  Date range: {data['timestamp'].min()} to {data['timestamp'].max()}")
    print(f"  Stations: {data['station_name'].nunique()}")
    print(f"  Pollutants: {data['pollutant_name'].nunique()}")
    print("\n  Pollutant breakdown:")
    for pollutant, count in data['pollutant_name'].value_counts().items():
        print(f"    {pollutant}: {count:,} measurements")
    
    return data

#### 2) Function for analysing overall trends.

CSV will be saved as yearly_averages.csv.

In [77]:
def analyse_overall_trends(data, report_dir):
    """
    Overall pollution trends across study period.
    
    Creates:
    - Yearly average comparison.
    - Monthly trends.
    - Overall statistics table.
    """
    
    print("\n" + "="*40)
    print("Overall pollutant trends")
    print("="*40)
    
    if data is None or data.empty:
        print("  Error: no data loaded.")
        return None
    
    # Calculate yearly averages per pollutant
    yearly_avg = data.groupby(['year', 'pollutant_name'])['value'].agg([
        'mean', 'median', 'std', 'count'
    ]).round(2)
    
    print("\nYearly averages by pollutant:")
    print(yearly_avg)
    
    # Save to CSV yearly_averages.csv under report/detailed_analysis
    os.makedirs(report_dir, exist_ok=True)
    yearly_avg.to_csv(report_dir / 'yearly_averages.csv')
    
    # Create visualisation with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Yearly averages bar chart
    yearly_pivot = data.groupby(['year', 'pollutant_name'])['value'].mean().unstack()
    yearly_pivot.plot(kind='bar', ax=axes[0], width=0.8, alpha=0.8)
    axes[0].set_xlabel('Year', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Average concentration', fontsize=12, fontweight='bold')
    axes[0].set_title('Yearly average pollution levels by pollutant', 
                     fontsize=13, fontweight='bold')
    axes[0].legend(title='Pollutant', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0].grid(axis='y', alpha=0.3)
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
    
    # Plot 2: Monthly trend across all years
    monthly_avg = data.groupby(['month_name', 'pollutant_name'])['value'].mean().unstack()
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    monthly_avg = monthly_avg.reindex(month_order)
    monthly_avg.plot(ax=axes[1], marker='o', linewidth=2)
    axes[1].set_xlabel('Month', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Average concentration', fontsize=12, fontweight='bold')
    axes[1].set_title('Monthly average pollution levels (all years combined)', 
                     fontsize=13, fontweight='bold')
    axes[1].legend(title='Pollutant', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1].grid(alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(report_dir / 'overall_trends.png', dpi=300, bbox_inches='tight')
    print(f"\n  Visualisation saved: overall_trends.png")
    plt.close()
    
    return yearly_avg

#### 3) Analyse specific pollutant.

Here I will do full analysis on the pollutants I fetched:
1. All pollutants at optimised/year_month/*.
2. The pollutants UK have limits - uk_pollutant_limits.csv.
3. And also pollutants common between DEFRA/LAQN: NO2, PM10, PM2.5, CO, SO2, O3.

The PNG visualisation will be saved as: (pollutant_name)_analysis.png.
Report/detailed_analysis path.

In [78]:
def analyse_pollutant_specific(data, pollutant, report_dir):
    """
    Deep dive into a specific pollutant.
    
    Parameters:
        pollutant : str - pollutant name (e.g., 'NO2', 'PM25').
            
    Creates:
        - Time series plot.
        - Distribution histogram.
        - Hourly pattern.
        - Statistics summary.
    """
    print(f"\n" + "="*40)
    print(f"Pollutant specific analysis: {pollutant}")
    print("="*40)
    
    if data is None or data.empty:
        print("  Error: no data loaded.")
        return None
    
    # Filter for this pollutant
    poll_data = data[data['pollutant_name'] == pollutant].copy()
    
    if poll_data.empty:
        print(f"  Error: no data found for {pollutant}.")
        return None
    
    # Remove NaN values before analysis
    poll_data_clean = poll_data[poll_data['value'].notna()].copy()
    
    if poll_data_clean.empty:
        print(f"  Warning: all values are NaN for {pollutant}, skipping analysis.")
        return None
    
    print(f"\n  Total measurements: {len(poll_data):,}")
    print(f"  Valid (non-NaN) measurements: {len(poll_data_clean):,}")
    print(f"  Date range: {poll_data_clean['timestamp'].min()} to {poll_data_clean['timestamp'].max()}")
    print(f"  Stations: {poll_data_clean['station_name'].nunique()}")
    
    # Statistics
    stats = poll_data_clean['value'].describe()
    print(f"\n  Concentration statistics for {pollutant}:")
    print(f"    Mean: {stats['mean']:.2f}")
    print(f"    Median: {stats['50%']:.2f}")
    print(f"    Std dev: {stats['std']:.2f}")
    print(f"    Min: {stats['min']:.2f}")
    print(f"    Max: {stats['max']:.2f}")
    print(f"    25th percentile: {stats['25%']:.2f}")
    print(f"    75th percentile: {stats['75%']:.2f}")
    
    # Create 4 panel visualisation
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Panel 1: Time series (daily average)
    daily_avg = poll_data_clean.groupby(poll_data_clean['timestamp'].dt.date)['value'].mean()
    axes[0, 0].plot(daily_avg.index, daily_avg.values, linewidth=1, alpha=0.7)
    axes[0, 0].set_xlabel('Date', fontsize=11, fontweight='bold')
    axes[0, 0].set_ylabel(f'{pollutant} concentration', fontsize=11, fontweight='bold')
    axes[0, 0].set_title(f'Daily average {pollutant} levels (2023-2025)', 
                        fontsize=12, fontweight='bold')
    axes[0, 0].grid(alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Panel 2: Distribution histogram (only non-NaN values)
    axes[0, 1].hist(poll_data_clean['value'], bins=50, color='steelblue', 
                   alpha=0.7, edgecolor='black')
    axes[0, 1].axvline(poll_data_clean['value'].mean(), color='red', linestyle='--', 
                      linewidth=2, label=f'Mean: {poll_data_clean["value"].mean():.1f}')
    axes[0, 1].axvline(poll_data_clean['value'].median(), color='orange', linestyle='--', 
                      linewidth=2, label=f'Median: {poll_data_clean["value"].median():.1f}')
    axes[0, 1].set_xlabel(f'{pollutant} concentration', fontsize=11, fontweight='bold')
    axes[0, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
    axes[0, 1].set_title(f'{pollutant} distribution', fontsize=12, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3, axis='y')
    
    # Panel 3: Hourly pattern
    hourly_avg = poll_data_clean.groupby('hour')['value'].mean()
    axes[1, 0].plot(hourly_avg.index, hourly_avg.values, marker='o', 
                   linewidth=2, color='forestgreen')
    axes[1, 0].set_xlabel('Hour of day', fontsize=11, fontweight='bold')
    axes[1, 0].set_ylabel(f'Average {pollutant}', fontsize=11, fontweight='bold')
    axes[1, 0].set_title(f'Average {pollutant} by hour of day', 
                        fontsize=12, fontweight='bold')
    axes[1, 0].set_xticks(range(0, 24, 2))
    axes[1, 0].grid(alpha=0.3)
    
    # Panel 4: Monthly boxplot
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    poll_data_clean['month_name'] = pd.Categorical(poll_data_clean['month_name'], 
                                                   categories=month_order, ordered=True)
    poll_data_clean.boxplot(column='value', by='month_name', ax=axes[1, 1])
    axes[1, 1].set_xlabel('Month', fontsize=11, fontweight='bold')
    axes[1, 1].set_ylabel(f'{pollutant} concentration', fontsize=11, fontweight='bold')
    axes[1, 1].set_title(f'Monthly {pollutant} distribution', 
                        fontsize=12, fontweight='bold')
    axes[1, 1].get_figure().suptitle('')  # Remove default title
    plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)
    
    plt.tight_layout()
    safe_name = pollutant.replace('.', '_').replace('/', '_').replace(' ', '_')
    plt.savefig(report_dir / f'{safe_name}_analysis.png', dpi=300, bbox_inches='tight')
    print(f"\n  Visualisation saved: {safe_name}_analysis.png")
    plt.close()
    
    return poll_data_clean

#### 4) Analyse seasonal patterns created for checking seasonal patterns.

Function creates season definition.
Calculates seasonal averages.
And visualises it.
Report/detailed_analysis folder and image name: seasonal_patterns.png.

In [79]:
def analyse_seasonal_patterns(data, report_dir):
    """
    Seasonal variation analysis.
    
    Creates:
    - Season definitions (Winter, Spring, Summer, Autumn).
    - Seasonal averages.
    - Visualisation.
    """
    
    print("\n" + "="*40)
    print("Seasonal pattern analysis")
    print("="*40)
    
    if data is None or data.empty:
        print("  Error: no data loaded.")
        return None
    
    # Define seasons
    def assign_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Autumn'
    
    data_copy = data.copy()
    data_copy['season'] = data_copy['month'].apply(assign_season)
    
    # Calculate seasonal averages
    seasonal_avg = data_copy.groupby(['season', 'pollutant_name'])['value'].mean().unstack()
    season_order = ['Winter', 'Spring', 'Summer', 'Autumn']
    seasonal_avg = seasonal_avg.reindex(season_order)
    
    print("\nSeasonal averages:")
    print(seasonal_avg.round(2))
    
    # Save
    seasonal_avg.to_csv(report_dir / 'seasonal_averages.csv')
    
    # Visualise
    fig, ax = plt.subplots(figsize=(12, 7))
    seasonal_avg.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
    ax.set_xlabel('Season', fontsize=12, fontweight='bold')
    ax.set_ylabel('Average concentration', fontsize=12, fontweight='bold')
    ax.set_title('Seasonal pollution patterns by pollutant', 
                fontsize=13, fontweight='bold')
    ax.legend(title='Pollutant', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(axis='y', alpha=0.3)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    
    plt.tight_layout()
    plt.savefig(report_dir / 'seasonal_patterns.png', dpi=300, bbox_inches='tight')
    print(f"\n  Visualisation saved: seasonal_patterns.png")
    plt.close()
    
    return seasonal_avg

#### 5) Analyse limit exceedances function checks the uk_pollutant_limits.csv.

1. Requires pollutant_limits.csv with columns: pollutant_std, limit, concentration measured as, objective.
2. Report/detailed_analysis path creates file called: limit_exceedances.csv.
   - Column structure of the file: pollutant, objective, averaging_period, limit_value, total_measurements, exceedances, exceedance_pct.

In [80]:
def analyse_limit_exceedances(data, report_dir, limits_path):
    """
    Analysis of UK legal limit exceedances.
    
    Matches pollutant names in data with standard codes in UK limits CSV.
    
    Creates:
        - Exceedance counts per pollutant.
        - Percentage of measurements exceeding limits.
        - Comparison against UK legal objectives.
    """
    print("\n" + "="*40)
    print("UK legal limit exceedance analysis")
    print("="*40)
    
    if data is None or data.empty:
        print("  Error: no data loaded.")
        return None
    
    # Load limits
    try:
        limits_df = pd.read_csv(limits_path)
        print(f"\n  Loaded {len(limits_df)} UK limit standards from: {limits_path.name}")
    except Exception as e:
        print(f"  Error: couldn't load limits file: {e}")
        return None
    
    # LAQN uses standard codes directly (NO2, PM10, PM25, etc.)
    # Map PM2.5 variations
    pollutant_mapping = {
        'PM2.5': 'PM25',
        'PM25': 'PM25',
        'NO2': 'NO2',
        'NO': 'NO',
        'NOx': 'NOx',
        'SO2': 'SO2',
        'O3': 'O3',
        'PM10': 'PM10',
        'CO': 'CO'
    }
    
    # Prepare results storage
    exceedance_results = []
    pollutants_checked = 0
    pollutants_with_limits = 0
    
    # For each pollutant in data
    for pollutant in data['pollutant_name'].unique():
        pollutants_checked += 1
        
        # Map to standard code (most LAQN codes are already standard)
        std_code = pollutant_mapping.get(pollutant, pollutant)
        
        # Get limits for this pollutant
        limits = limits_df[limits_df['pollutant_std'] == std_code]
        
        if limits.empty:
            continue  # Skip if no limits defined
        
        pollutants_with_limits += 1
        
        # Get clean data (remove NaN)
        poll_data = data[data['pollutant_name'] == pollutant].copy()
        poll_data = poll_data[poll_data['value'].notna()]
        
        if poll_data.empty:
            print(f"\n  {pollutant}: No valid data to check.")
            continue
        
        print(f"\n  Checking {pollutant} ({std_code}):")
        
        # Check each limit type for this pollutant
        for _, limit_row in limits.iterrows():
            limit_value = limit_row['limit']
            averaging = limit_row['concentration measured as']
            objective = limit_row['objective']
            unit = limit_row['unit']
            
            # Handle unit conversion if needed
            data_values = poll_data['value'].copy()
            if unit == 'mg/m3':
                # Convert µg/m3 to mg/m3 if needed (divide by 1000)
                # Assuming data is in µg/m3, convert to mg/m3
                data_values = data_values / 1000.0
                print(f"    Note: converted values from µg/m³ to mg/m³ for comparison.")
            
            # Calculate exceedances
            exceedances = (data_values > limit_value).sum()
            total = len(data_values)
            pct = (exceedances / total * 100) if total > 0 else 0
            
            exceedance_results.append({
                'pollutant': pollutant,
                'pollutant_code': std_code,
                'objective': objective,
                'averaging_period': averaging,
                'limit_value': limit_value,
                'unit': unit,
                'total_measurements': total,
                'exceedances': exceedances,
                'exceedance_pct': round(pct, 2)
            })
            
            # Report results
            if exceedances > 0:
                print(f"    {averaging}: {exceedances:,} / {total:,} ({pct:.1f}%) EXCEED {limit_value} {unit}")
            else:
                print(f"    {averaging}: All measurements within limit ({limit_value} {unit}).")
    
    # Summary
    print(f"\n  Summary:")
    print(f"    Total pollutants in dataset: {pollutants_checked}")
    print(f"    Pollutants with UK limits: {pollutants_with_limits}")
    
    if exceedance_results:
        exceedance_df = pd.DataFrame(exceedance_results)
        exceedance_df.to_csv(report_dir / 'limit_exceedances.csv', index=False)
        print(f"    Results saved: limit_exceedances.csv")
        
        # Print overall exceedance summary
        total_exceedances = exceedance_df['exceedances'].sum()
        total_checks = exceedance_df['total_measurements'].sum()
        print(f"\n  Overall: {total_exceedances:,} exceedances out of {total_checks:,} measurements checked.")
        
        return exceedance_df
    else:
        print("\n  No matching limits found for any pollutants in dataset.")
        return None

#### 6) All analyses.

Finally function for run_full_analysis for all the descriptions.

In [81]:
def run_full_analysis(base_dir, csv_output_path=None, sample_size=None):
    """
    Run complete analysis pipeline.

    Parameters:
        base_dir : str or Path
            Path to optimised directory.
        csv_output_path : str or Path, optional
            Path to pollutant_limits.csv.
        sample_size : int, optional
            Number of files to load (None = all).
            
    Returns:
        data : DataFrame with loaded pollution data.
    """

    print("\n" + "="*40)
    print("Starting full pollution analysis")
    print("="*40)
    
    # Load data
    data = load_pollution_data(base_dir, sample_size=sample_size)
    
    if data.empty:
        print("\n  Error: no data loaded, cannot continue.")
        return None
    
    # Run all analyses
    analyse_overall_trends(data, report_dir)
    
    # Analyze each pollutant found in data
    for pollutant in data['pollutant_name'].unique():
        analyse_pollutant_specific(data, pollutant, report_dir)
    
    analyse_seasonal_patterns(data, report_dir)
    
    if csv_output_path and Path(csv_output_path).exists():
        analyse_limit_exceedances(data, report_dir, Path(csv_output_path))
    
    print("\n" + "="*40)
    print("Analysis complete")
    print("="*40)
    print(f"\nAll results saved to: {report_dir}")
    print("\nGenerated files:")
    for f in sorted(report_dir.iterdir()):
        print(f"  - {f.name}")
    print("="*40)
    
    return data

In [83]:
# # Run full analysis on LAQN data
# # Start with sample of 100 files for quick exploration
# data = run_full_analysis(
#     base_dir=base_dir,
#     csv_output_path=csv_output_path,
#     sample_size=100
# )

# Once exploration is complete, run on full dataset:
data = run_full_analysis(
    base_dir=base_dir,
    csv_output_path=csv_output_path,
    sample_size=None
)


Starting full pollution analysis

  Loaded 3,446,208 measurements from 4932 files.
  Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
  Stations: 64
  Pollutants: 6

  Pollutant breakdown:
    NO2: 1,417,752 measurements
    PM10: 1,026,456 measurements
    PM2.5: 586,944 measurements
    O3: 268,320 measurements
    SO2: 97,824 measurements
    CO: 48,912 measurements

Overall pollutant trends

Yearly averages by pollutant:
                      mean  median    std   count
year pollutant_name                              
2023 CO               0.24     0.2   0.19   16242
     NO2             24.92    20.0  18.67  436830
     O3              46.28    46.8  24.13   77787
     PM10            16.69    14.0  12.40  317643
     PM2.5            8.81     7.0   7.08  154094
     SO2              1.21     0.8   1.23   27604
2024 CO               0.16     0.1   0.17   14592
     NO2             22.78    19.0  15.97  444363
     O3              47.38    47.9  22.97   76265
     PM10     

    ========================================
    Starting full pollution analysis
    ========================================

    Loaded 3,446,208 measurements from 4932 files.
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 64
    Pollutants: 6

    Pollutant breakdown:
        NO2: 1,417,752 measurements
        PM10: 1,026,456 measurements
        PM2.5: 586,944 measurements
        O3: 268,320 measurements
        SO2: 97,824 measurements
        CO: 48,912 measurements

    ========================================
    Overall pollutant trends
    ========================================

    Yearly averages by pollutant:
                        mean  median    std   count
    year pollutant_name                              
    2023 CO               0.24     0.2   0.19   16242
        NO2             24.92    20.0  18.67  436830
        O3              46.28    46.8  24.13   77787
        PM10            16.69    14.0  12.40  317643
        PM2.5            8.81     7.0   7.08  154094
        SO2              1.21     0.8   1.23   27604
    2024 CO               0.16     0.1   0.17   14592
        NO2             22.78    19.0  15.97  444363
        O3              47.38    47.9  22.97   76265
        PM10            16.32    14.0  11.61  312108
        PM2.5            8.42     6.9   7.33  174697
        SO2              1.17     0.8   1.74   23860
    2025 CO               0.20     0.2   0.18   12731
        NO2             22.19    17.9  16.59  385825
        O3              49.73    49.7  24.70   66860
        PM10            18.94    15.9  13.33  266395
        PM2.5           10.63     8.0   8.96  153768
        SO2              2.18     1.4   5.55   19753

    Visualisation saved: overall_trends.png

    ========================================
    Pollutant specific analysis: PM10
    ========================================

    Total measurements: 1,026,456
    Valid (non-NaN) measurements: 896,146
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 42

    Concentration statistics for PM10:
        Mean: 17.23
        Median: 14.40
        Std dev: 12.47
        Min: 0.00
        Max: 759.00
        25th percentile: 9.50
        75th percentile: 21.40

    Visualisation saved: PM10_analysis.png

    ========================================
    Pollutant specific analysis: NO2
    ========================================

    Total measurements: 1,417,752
    Valid (non-NaN) measurements: 1,267,018
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 58

    Concentration statistics for NO2:
        Mean: 23.34
        Median: 19.00
        Std dev: 17.17
        Min: 0.00
        Max: 376.30
        25th percentile: 10.50
        75th percentile: 31.90

    Visualisation saved: NO2_analysis.png

    ========================================
    Pollutant specific analysis: PM2.5
    ========================================

    Total measurements: 586,944
    Valid (non-NaN) measurements: 482,559
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 24

    Concentration statistics for PM2.5:
        Mean: 9.25
        Median: 7.00
        Std dev: 7.87
        Min: 0.00
        Max: 909.00
        25th percentile: 4.80
        75th percentile: 11.00

    Visualisation saved: PM2_5_analysis.png

    ========================================
    Pollutant specific analysis: SO2
    ========================================

    Total measurements: 97,824
    Valid (non-NaN) measurements: 71,217
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 4

    Concentration statistics for SO2:
        Mean: 1.47
        Median: 1.10
        Std dev: 3.21
        Min: 0.00
        Max: 271.40
        25th percentile: 0.50
        75th percentile: 1.80

    Visualisation saved: SO2_analysis.png

    ========================================
    Pollutant specific analysis: CO
    ========================================

    Total measurements: 48,912
    Valid (non-NaN) measurements: 43,565
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 2

    Concentration statistics for CO:
        Mean: 0.20
        Median: 0.20
        Std dev: 0.19
        Min: 0.00
        Max: 4.90
        25th percentile: 0.10
        75th percentile: 0.30

    Visualisation saved: CO_analysis.png

    ========================================
    Pollutant specific analysis: O3
    ========================================

    Total measurements: 268,320
    Valid (non-NaN) measurements: 220,912
    Date range: 2023-01-01 00:00:00 to 2025-11-18 23:00:00
    Stations: 11

    Concentration statistics for O3:
        Mean: 47.71
        Median: 48.10
        Std dev: 23.95
        Min: 0.00
        Max: 198.60
        25th percentile: 31.10
        75th percentile: 63.50

    Visualisation saved: O3_analysis.png

    ========================================
    Seasonal pattern analysis
    ========================================

    Seasonal averages:
    pollutant_name    CO    NO2     O3   PM10  PM2.5   SO2
    season                                                
    Winter          0.28  28.14  39.20  17.10  10.09  1.63
    Spring          0.16  23.01  57.33  18.74  10.58  1.52
    Summer          0.15  18.25  53.14  16.13   7.46  1.41
    Autumn          0.22  24.69  39.80  16.91   8.90  1.34

    Visualisation saved: seasonal_patterns.png

    ========================================
    UK legal limit exceedance analysis
    ========================================

    Loaded 12 UK limit standards from: uk_pollutant_limits.csv

    Checking PM10 (PM10):
        24 hour mean: 18,787 / 896,146 (2.1%) EXCEED 50.0 µg/m3
        annual mean: 40,685 / 896,146 (4.5%) EXCEED 40.0 µg/m3

    Checking NO2 (NO2):
        annual mean: 194,871 / 1,267,018 (15.4%) EXCEED 40.0 µg/m3

    Checking SO2 (SO2):
        24 hour mean: 14 / 71,217 (0.0%) EXCEED 125.0 µg/m3

    Checking CO (CO):
        Note: converted values from µg/m³ to mg/m³ for comparison.
        maximum daily: All measurements within limit (10.0 mg/m3).

    Checking O3 (O3):
        8 hour mean: 3,677 / 220,912 (1.7%) EXCEED 100.0 µg/m3

    Summary:
        Total pollutants in dataset: 6
        Pollutants with UK limits: 5
        Results saved: limit_exceedances.csv

    Overall: 258,034 exceedances out of 3,395,004 measurements checked.

    ========================================
    Analysis complete
    ========================================

    All results saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/report/detailed_analysis

    Generated files:
    - CO_analysis.png
    - NO2_analysis.png
    - O3_analysis.png
    - PM10_analysis.png
    - PM2_5_analysis.png
    - SO2_analysis.png
    - limit_exceedances.csv
    - overall_trends.png
    - seasonal_averages.csv
    - seasonal_patterns.png
    - yearly_averages.csv
    ========================================